In [1]:
# Insert code here.
import pandas as pd
import numpy as np
import random
import re
import time
import datetime
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, neighbors
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
import torch
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
import matplotlib.pyplot as plt
import seaborn as sns
# from sentence_transformers import SentenceTransformer
# sent_encoder = SentenceTransformer('bert-base-nli-mean-tokens')

In [2]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")
torch.cuda.empty_cache()

There are 4 GPU(s) available.
We will use the GPU: GeForce RTX 2080 Ti


In [3]:
torch.cuda.empty_cache()
!nvidia-smi

Fri Oct 23 15:04:34 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.95.01    Driver Version: 440.95.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:02:00.0 Off |                  N/A |
| 35%   57C    P0    69W / 250W |     11MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:03:00.0 Off |                  N/A |
| 30%   26C    P0    70W / 250W |     11MiB / 11019MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [4]:
DATASET_PATH = '../datasets/nela-gt/nela10.csv'

In [5]:
df = pd.read_csv(DATASET_PATH)
print('Number of training sentences: {:,}\n'.format(df.shape[0]))
df.sample(10)

Number of training sentences: 57,157



,id,date,source,title,content,author,url,published,published_utc,collection_utc,Reliability
21604,tass--2019-02-04--Kremlin stresses only Venezu...,2019-02-04,tass,Kremlin stresses only Venezuelans themselves c...,"MOSCOW, February 4. /TASS/. Moscow has labeled...",NaN,http://tass.com/politics/1043139,2019-02-04 11:17:35+00:00,1549297055,1567549654,0
48761,intellihub--2019-10-04--Ukraine to review inve...,2019-10-04,intellihub,Ukraine to review investigation into Biden-lin...,After a week of non-stop “bombshell” leaks abo...,Zero Hedge,https://www.intellihub.com/ukraine-to-review-i...,2019-10-04 12:13:12+00:00,1570205592,1570633618,2
5577,cnbc--2019-02-04--Ocasio-Cortez and Warrens we...,2019-02-04,cnbc,Ocasio-Cortez and Warren's wealth tax plans ar...,"""I wish some economist would go and talk to th...",NaN,https://www.cnbc.com/2019/02/04/hassett-warren...,2019-02-04 13:47:00+00:00,1549306020,1567549705,0
11514,france24--2019-08-12--India death toll rises t...,2019-08-12,france24,India death toll rises to 144 as floods cut of...,"STR, AFP | Rescue workers carry a body as they...",NEWS WIRES,https://www.france24.com/en/20190812-india-flo...,2019-08-12 08:52:32+00:00,1565614352,1567534381,0
56494,vdare--2019-09-18--A Catholic Reader Reproves ...,2019-09-18,vdare,A Catholic Reader Reproves Matthew Schmitz For...,Re: This French-Sohrab Ahmari “Debate” Won’t G...,reader@vdare.com (VDARE.com Reader),https://vdare.com/letters/a-catholic-reader-re...,2019-09-18 01:36:59+00:00,1568785019,1569329981,2
50802,newswars--2019-06-27--Don Jr Endorses Infowars...,2019-06-27,newswars,Don Jr. Endorses Infowars’ “Clown World” Parad...,Donald Trump Jr. echoed Infowars’ Democrat Deb...,Infowars.com,https://www.newswars.com/don-jr-endorses-infow...,2019-06-27 02:06:31+00:00,1561615591,1567537924,2
51109,prisonplanet--2019-02-04--Politico Trump Is Ma...,2019-02-04,prisonplanet,Politico: Trump Is Making People Sick,CNN before lovemaking is not his idea of a tur...,admin,https://www.prisonplanet.com/politico-trump-is...,2019-02-04 11:26:15+00:00,1549297575,1567549732,2
3972,bbcuk--2019-06-27--Ex-Army translator wants to...,2019-06-27,bbcuk,Ex-Army translator wants to swap burgers for N...,An Afghan interpreter who risked his life help...,NaN,https://www.bbc.co.uk/news/uk-england-leiceste...,2019-06-27 14:05:23+00:00,1561658723,1567537912,0
12236,hotair--2019-06-27--Audio Trump accusers confi...,2019-06-27,hotair,Audio: Trump accuser’s confidants corroborate ...,The full NYT podcast is long at nearly 30 minu...,Allahpundit,https://hotair.com/archives/2019/06/27/audio-t...,2019-06-27 18:01:36+00:00,1561672896,1567537925,0
13041,mcclatchydc--2019-10-08--Justice Department la...,2019-10-08,mcclatchydc,Justice Department labels Devin Nunes’ claim o...,A talking point used by some leading Republica...,"<p><span class=""ng_byline_name"">By Kevin G. Ha...",https://www.mcclatchydc.com/news/crime/article...,"Tue, 08 Oct 2019 17:54:42 EDT",1570571682,1570842344,0


In [6]:
def count_words(text):
    try:
        return len(text.split())
    except:
        print(text)
        return None

In [7]:
def change_rel(num):
    if num == 2:
        return 1
    else:
        return 0

In [8]:
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = []

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = str(text)
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['content'] = df['content'].apply(clean_text)
df['content'] = df['content'].str.replace('\d+', '')
df['Reliability'] = df['Reliability'].apply(change_rel)

In [9]:
df['content'].apply(count_words).max()

15183

In [10]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['content'], df['Reliability'], test_size=0.2, stratify=df['Reliability'])

In [11]:
MAX_LENGTH = 511
posts = train_x.values
categories = train_y.values

In [12]:
encoded_dict = tokenizer.encode_plus(
                        posts[0],                      # Sentence to encode.
                        truncation=True,
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = MAX_LENGTH,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
encoded_dict

/home/tathagata.raha/miniconda3/envs/fastai/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1764: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


{'input_ids': tensor([[  101,  1037,  6715,  2038,  2042,  2218,  2005,  1037,  2329,  3232,
          2040,  2351,  2306,  2847,  1997,  2169,  2060,  1999,  2660, 17386,
          2239,  4557,  2001,  2718,  2011,  1037,  2482,  1999,  9300,  2006,
          2285,  1998,  4256,  5650,  6157,  2001,  2101,  2179,  2757,  2050,
          4101,  2326,  1997,  7401,  1998, 15060,  2001,  2218,  2012, 10093,
          3877, 13675, 14545, 24390,  1999, 19322,  2006,  5958, 19699,  9013,
          2094,  4775,  2577,  2056,  2027,  2106,  2062,  1999,  1996,  2261,
          2086,  2027,  2020,  2362,  2084,  2087,  2111,  2079,  1999,  1037,
          6480, 10760,  3232,  2761,  2013, 19322,  2333,  2000,  9300,  2197,
          2095,  5974,  2084,  2827,  6363,  2001,  2992,  3081,  1037,  2175,
          4636,  2033,  3931,  2275,  2039,  2011,  2720,  4557,  2827,  4043,
          2136, 26046,  4570, 26846,  2000,  2393,  1996,  6062,  9064,  2007,
          2151,  5366,  3378,  2007,  

In [13]:
input_ids = []
attention_masks = []

# For every sentence...
for sent in posts:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    try:
        encoded_dict = tokenizer.encode_plus(
                            sent,                      # Sentence to encode.
                            truncation=True,
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = MAX_LENGTH,           # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                       )
    except:
        print(sent)
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(categories)

In [14]:
dataset = TensorDataset(input_ids, attention_masks, labels)
train_size = int(0.875 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))


40,009 training samples
5,716 validation samples


In [15]:
# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 8

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )


In [16]:
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
import torch.nn as nn
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
# model = nn.DataParallel(model)
model.to(device)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [17]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))


The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [18]:
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [19]:
# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


In [20]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


In [21]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))



In [ ]:
seed_val = 42
torch.cuda.empty_cache()
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
#         print(loss)
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))
    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
    if epoch_i >= 1:
        inp = input()
        if inp.startswith('y'):
            break

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))



======== Epoch 1 / 4 ========
Training...
  Batch    40  of  5,002.    Elapsed: 0:00:12.
  Batch    80  of  5,002.    Elapsed: 0:00:25.
  Batch   120  of  5,002.    Elapsed: 0:00:37.
  Batch   160  of  5,002.    Elapsed: 0:00:50.
  Batch   200  of  5,002.    Elapsed: 0:01:02.
  Batch   240  of  5,002.    Elapsed: 0:01:15.
  Batch   280  of  5,002.    Elapsed: 0:01:27.
  Batch   320  of  5,002.    Elapsed: 0:01:40.
  Batch   360  of  5,002.    Elapsed: 0:01:52.
  Batch   400  of  5,002.    Elapsed: 0:02:05.
  Batch   440  of  5,002.    Elapsed: 0:02:18.
  Batch   480  of  5,002.    Elapsed: 0:02:30.
  Batch   520  of  5,002.    Elapsed: 0:02:43.
  Batch   560  of  5,002.    Elapsed: 0:02:56.
  Batch   600  of  5,002.    Elapsed: 0:03:08.
  Batch   640  of  5,002.    Elapsed: 0:03:21.
  Batch   680  of  5,002.    Elapsed: 0:03:34.
  Batch   720  of  5,002.    Elapsed: 0:03:46.
  Batch   760  of  5,002.    Elapsed: 0:03:59.
  Batch   800  of  5,002.    Elapsed: 0:04:12.
  Batch   840  of

 no



======== Epoch 3 / 4 ========
Training...
  Batch    40  of  5,002.    Elapsed: 0:00:12.
  Batch    80  of  5,002.    Elapsed: 0:00:25.
  Batch   120  of  5,002.    Elapsed: 0:00:37.
  Batch   160  of  5,002.    Elapsed: 0:00:49.
  Batch   200  of  5,002.    Elapsed: 0:01:02.
  Batch   240  of  5,002.    Elapsed: 0:01:14.
  Batch   280  of  5,002.    Elapsed: 0:01:27.
  Batch   320  of  5,002.    Elapsed: 0:01:40.
  Batch   360  of  5,002.    Elapsed: 0:01:52.
  Batch   400  of  5,002.    Elapsed: 0:02:05.
  Batch   440  of  5,002.    Elapsed: 0:02:18.
  Batch   480  of  5,002.    Elapsed: 0:02:30.
  Batch   520  of  5,002.    Elapsed: 0:02:43.
  Batch   560  of  5,002.    Elapsed: 0:02:56.
  Batch   600  of  5,002.    Elapsed: 0:03:08.
  Batch   640  of  5,002.    Elapsed: 0:03:21.
  Batch   680  of  5,002.    Elapsed: 0:03:34.
  Batch   720  of  5,002.    Elapsed: 0:03:46.
  Batch   760  of  5,002.    Elapsed: 0:03:59.
  Batch   800  of  5,002.    Elapsed: 0:04:12.
  Batch   840  of